In [ ]:
import napari
from pathlib import Path
from trackedit.motile_overwrites import *
from trackedit.TrackEdit_functions import *
from trackedit.DatabaseHandler import DatabaseHandler
from motile_tracker.data_model.actions import AddEdges, DeleteNodes, DeleteEdges, AddNodes


#**********INPUTS*********
working_directory = Path('/home/teun.huijben/Documents/data/Akila/20241003/neuromast4_t851/adjusted/')
db_filename_old = 'data.db'
data_shape_full = [851,73,1024,1024]      #T,(Z),Y,X       (851,73,1024,1024)
scale = (4,1,1)
layer_name = 'ultrack'
allow_overwrite = True      #overwrite existing database/changelog
#*************************

DB_handler = DatabaseHandler(
            db_filename_old = db_filename_old,
            working_directory = working_directory,
            data_shape_full = data_shape_full,
            z_scale = scale[0],
            name = 'ultrack',
            allow_overwrite = allow_overwrite)

#overwrite some motile functions
DeleteNodes._apply = create_db_delete_nodes(DB_handler)
DeleteEdges._apply = create_db_delete_edges(DB_handler)
AddEdges._apply = create_db_add_edges(DB_handler)
AddNodes._apply = create_db_add_nodes(DB_handler)
TracksViewer._refresh = create_tracks_viewer_and_segments_refresh(layer_name=layer_name)


#open napari with TrackEdit
viewer = napari.Viewer()
trackeditclass = TrackEditClass(viewer, databasehandler = DB_handler)
viewer.dims.ndisplay = 3    #3D view

# Red flags

In [ ]:
DB_handler.df

In [ ]:
import numpy as np
# def find_red_flags_in_df(df: pd.DataFrame) -> pd.DataFrame:
#     """
#     Identify tracking red flags, i.e. when cell tracks appear ('added') or disappear ('removed')
#     from one timepoint to the next, and include the corresponding 'id' field from the original dataframe.
    
#     A track is flagged as 'added' if it is present at the current timepoint but not in the previous one,
#     and flagged as 'removed' if it is present at the current timepoint but not in the next one.
    
#     Parameters
#     ----------
#     df : pd.DataFrame
#         DataFrame containing at least 't' (timepoint), 'track_id', and 'id' columns.
    
#     Returns
#     -------
#     pd.DataFrame
#         DataFrame with columns: 'Timepoint', 'track_id', 'event', and 'id'.
#     """
#     # Ensure the timepoints are continuous even if some are missing
#     time_range = range(df['t'].min(), df['t'].max() + 1)
    
#     # Group by timepoint and create a Series mapping each time to its set of track_ids.
#     track_sets = df.groupby('t')['track_id'].agg(set).reindex(time_range, fill_value=set())
    
#     # Precompute a mapping from (timepoint, track_id) to the corresponding "id" field.
#     # Assumes that there is only one row per (t, track_id).
#     mapping = df.set_index(['t', 'track_id'])['id'].to_dict()
    
#     events = []
#     timepoints = list(time_range)
#     for i, t in enumerate(timepoints):
#         current = track_sets[t]
#         # For the first timepoint, treat the previous set as the current set.
#         prev = track_sets[timepoints[i - 1]] if i > 0 else current
#         # For the last timepoint, treat the next set as the current set.
#         nxt = track_sets[timepoints[i + 1]] if i < len(timepoints) - 1 else current
        
#         # Tracks added at time t (present now but not in the previous timepoint)
#         for track in current - prev:
#             events.append({
#                 't': t,
#                 'track_id': track,
#                 'id': mapping.get((t, track)),
#                 'event': 'added',
#             })
#         # Tracks removed at time t (present now but not in the next timepoint)
#         for track in current - nxt:
#             events.append({
#                 't': t,
#                 'track_id': track,
#                 'id': mapping.get((t, track)),
#                 'event': 'removed',
#             })
    
#     return pd.DataFrame(events)

# df = DB_handler.db_to_df(entire_database=True)
red_flags = DB_handler.find_all_red_flags()
display(red_flags)
print(red_flags.shape)

In [ ]:
print(len(red_flags))
print(' ')
print(red_flags.loc[[0]])

df_rf = red_flags.loc[[0]]
df_rf.iloc[0].t

ToDo: 
- want red flags of entire dataset!